# **Install Dependencies & Mount Google Drive**

In [ ]:
import os
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import drive

#  Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Configuration**

In [ ]:
MODEL_NAME = "unet"  # Ensure this matches the trained model name
SAVE_MODEL_PATH = "/content/drive/MyDrive/Medical_Image_Segmentation_Pipeline/save_trained_models"  # Folder where trained models are saved
INPUT_IMAGE_PATH = "/content/drive/MyDrive/Medical_Image_Segmentation_Pipeline/input_images"  # Folder containing images for prediction
PREDICTED_MASK_PATH = "/content/drive/MyDrive/Medical_Image_Segmentation_Pipeline/predicted_masks"  # Folder where masks will be saved
IMG_SIZE = 256
# Ensure the predicted masks folder exists
os.makedirs(PREDICTED_MASK_PATH, exist_ok=True)

# **Load Trained Model**

In [ ]:
import tensorflow as tf

# Re-define dice loss (needed for loading the model)
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1-(2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

model_path = os.path.join(SAVE_MODEL_PATH, f"{MODEL_NAME}_trained.h5")
# Load model with custom loss function
model = tf.keras.models.load_model(model_path, custom_objects={"dice_loss": dice_loss})

print(f"Model '{MODEL_NAME}' loaded successfully!")


Model 'unet' loaded successfully!


# **Load Input Images**

In [ ]:
input_images = sorted(os.listdir(INPUT_IMAGE_PATH))
if not input_images:
    raise FileNotFoundError(f"No input images found in {INPUT_IMAGE_PATH}")

print(f"Found {len(input_images)} input images for prediction.")


Found 2 input images for prediction.


# **Function to Load & Preprocess Image**

In [ ]:
def load_and_preprocess_image(image_path):
    """Load and preprocess image for model prediction."""
    image = cv2.imread(image_path)  # Load with OpenCV (to preserve original size)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    # Save original size
    orig_height, orig_width = image.shape[:2]
    # Resize for model input using IMG_SIZE variable and normalize pixel values
    image_resized = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image_resized = image_resized / 255.0
    return image_resized, orig_height, orig_width


# **Predict & Save Masks**

In [ ]:
for img_name in input_images:
    img_path = os.path.join(INPUT_IMAGE_PATH, img_name)

    # Load & preprocess image using updated function
    image, orig_height, orig_width = load_and_preprocess_image(img_path)
    image_input = np.expand_dims(image, axis=0)  # Add batch dimension

    # Predict mask using the model
    predicted_mask = model.predict(image_input)[0]
    # Convert prediction to binary mask using threshold 0.5
    binary_mask = (predicted_mask > 0.9).astype(np.uint8) * 255

    # Resize mask back to original image size using nearest-neighbor interpolation
    predicted_mask_resized = cv2.resize(binary_mask, (orig_width, orig_height), interpolation=cv2.INTER_NEAREST)

    # Save predicted mask (saving part kept unchanged)
    mask_save_path = os.path.join(PREDICTED_MASK_PATH, img_name)
    cv2.imwrite(mask_save_path, predicted_mask_resized)

    print(f"Saved predicted mask: {mask_save_path}")

print("🎉 *Prediction Complete! Masks saved successfully.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
Saved predicted mask: /content/drive/MyDrive/Medical_Image_Segmentation_Pipeline/predicted_masks/b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
Saved predicted mask: /content/drive/MyDrive/Medical_Image_Segmentation_Pipeline/predicted_masks/c.jpg
🎉 *Prediction Complete! Masks saved successfully.
